# Henting, lesing, og rensing av Trondheim forecast

### Utviklingsmiljøet:
Arbeidet med utviklingen av gruppe 84 sin miljødataaplikasjon har hovedsakelig foregått på brukeren Edi Galane sin github og VScode. De andre brukerene har bidrat med å skrive ulike deler av koden og funksjonaliteten, men endringene til selve koden har blitt utført gjennom Edi Galane din datamaskin. 

### Valg av API:
Datagrunnlaget for dette prosjektet har vært basert på MET.no (Meteorologisk institutt) sin åpne Locationforecast 2.0.

Det meteorologiske instituttet er en svært pålitelig kilde og Locationforecast 2.0 er et av deres høyest kurerte produkter. Til tross for at databasen er av høyest kvalitet, så er det nødvendig å utføre utforskende metoder for å sikre dens kvalitet og hente ut kun de nødvendige informasjonene. 

#### Import av nødvendige moduler og klasser:

In [1]:
import os
print("Jeg kjører notebooken fra:", os.getcwd())


Jeg kjører notebooken fra: /Users/edigalane/kode/MiljoDataProsjekt/notebooks


In [2]:
import os
import sys

# finn sti til src-mappen
sys.path.append(os.path.abspath("../src"))

# import av egne moduler og klasser
from fetch_data import FetchData
from data_reader import DataLeser
from data_cleaning import DataRensing


#### Henting og lagring av nødvendig rådata:

In [3]:
henter = FetchData()
df = henter.lagre_trondheim_forecast()

- lagre_trondheim_forecast() er en metode fra FetchData som har innebygd metoder for å hente en JSON-fil fra Locationforecast2.0 og flate den ut til en CSV-fil basert på parametrene for byen Trondheim.

#### Les og beskriv:

In [4]:
leser = DataLeser(data_dir="../data")
df_raw = leser.les_csv("trondheim_forecast_uncleaned.csv")  
leser.beskriv_dataframe(df_raw, "trondheim_forecast_uncleaned.csv")


--- trondheim_forecast_uncleaned.csv ---
Antall rader: 84
Antall kolonner: 12
De 5 første radene:
                   time  data_instant_details_air_pressure_at_sea_level  \
0  2025-05-27T02:00:00Z                                          1001.4   
1  2025-05-27T03:00:00Z                                          1001.5   
2  2025-05-27T04:00:00Z                                          1001.4   
3  2025-05-27T05:00:00Z                                          1001.6   
4  2025-05-27T06:00:00Z                                          1001.4   

   data_instant_details_air_temperature  \
0                                   5.6   
1                                   5.7   
2                                   7.2   
3                                   8.7   
4                                  10.1   

   data_instant_details_cloud_area_fraction  \
0                                       1.6   
1                                      10.8   
2                                      79.4   
3  

- Her ser vi de 5 første radene av det urensede datasettet. Neste steg er å velge ut de nødvendige kolonnene og rense dem.

#### Rense og lagre:
- Her tas data_rens metoden i bruk fra DataRensing klassen. Den inkluderer følgende metoder:
    - Brukerinteraktivitet for avrunding og filtrering av kolonner.
    - Bygging av en ny pandas DataFrame.
    - Fjerning av duplikater for å unngå overrepresentasjon av datapunkter. 
    - Median imputasjon for manglende verdier som bevarer semtraltendenser uten å påvirkes ekstreme verdier.
    - Bearbeiding av tidsstempler til lesbar datetime format.
    - Lagring av det nye pandas DataFrame til data mappen under "csv".
           

In [5]:
renser = DataRensing(df_raw)

# Brukerinput
temp_input = input("Filtrer ut temperatur over (la stå tom for å hoppe over): ")
avrund_input = input("Vil du avrunde en kolonne? (la stå tom for å hoppe over): ")
desimal_input = input("Hvor mange desimaler? (standard = 1): ")

# Konversjon av input
temp_grense = float(temp_input) if temp_input else None
avrund_kol = avrund_input if avrund_input else None
desimaler = int(desimal_input) if desimal_input else 1

# Bruker data_rens metoden
df_cleaned = renser.data_rens( 
    temp_grense=temp_grense,
    avrund_kol=avrund_kol,
    desimaler=desimaler,
    filnavn="trondheim_forecast_cleaned.csv"
)

# Beskriv det rensede datasettet
leser.beskriv_dataframe(df_cleaned,"trondheim_forecast_cleaned.csv")

Ny dataframe bygget
Kolonner: ['Tid', 'Temperatur', 'Fuktighet', 'Trykk', 'Vindhastighet']
Antall duplikater: 0, Antall manglende verdier: 0
MAnglende verdier og fuplikater håndtert
Tid formatert
Data lagret til /Users/edigalane/kode/MiljoDataProsjekt/data/trondheim_forecast_cleaned.csv
Data er ferdigrenset

--- trondheim_forecast_cleaned.csv ---
Antall rader: 84
Antall kolonner: 5
De 5 første radene:
                Tid  Temperatur  Fuktighet   Trykk  Vindhastighet
0  27.05.25 - 02:00         5.6       91.4  1001.4            2.0
1  27.05.25 - 03:00         5.7       90.9  1001.5            1.8
2  27.05.25 - 04:00         7.2       86.7  1001.4            0.6
3  27.05.25 - 05:00         8.7       80.2  1001.6            0.7
4  27.05.25 - 06:00        10.1       70.2  1001.4            1.2


Etter rensingen har vi ulike transformasjoner:
- Kolonnene [Tid, Temperatur, Fuktighet, Trykk, Vindhastighet] er bevart og omdøpt til lesbare navn. 
- Duplikater fjernes for å unngå at analysen blir skjevfordelt eller vektet feil.
- Manglende verdier inputeres som medianen siden denne ikke påvirkes av ekstreme uteliggere som kan oppdages i værdata grunnet dens urregelmessighet
- Resultatet er en DataFrame som lettere kan analyseres og modelleres. 

#### SQL-spørring
Vi har også implementert en valgfritt sql_utforsk metode i DataLeser. Men hva gjør SQL egentlig?
- Filtrerer data raskt basert på bestemte kriterier.
- Velger ut spesifikke kolonner og rader.
- Utfører aggregering som gjennomsnitt, maks, min, grupperinger osv...

In [6]:
sql_query = """
SELECT
    Tid,
    Temperatur,
    Fuktighet,
    Trykk,
    Vindhastighet
FROM df
WHERE Temperatur > 10
ORDER BY Tid ASC
LIMIT 10
"""

print("Hent de 10 første radene med temperatur > 10 grader celcius")
try:
    df_sql = leser.sql_utforsk(df_cleaned, sql_query)
    display(df_sql)
except ValueError as e:
    print(f"Feil i SQL-spørring: {e}")

Hent de 10 første radene med temperatur > 10 grader celcius


,Tid,Temperatur,Fuktighet,Trykk,Vindhastighet
0,01.06.25 - 06:00,10.7,72.1,1009.7,2.4
1,01.06.25 - 12:00,18.6,51.8,1005.2,3.0
2,01.06.25 - 18:00,16.9,67.1,1002.6,2.7
3,02.06.25 - 00:00,11.7,93.6,1002.3,2.1
4,02.06.25 - 06:00,11.9,90.8,1001.1,1.5
5,02.06.25 - 12:00,15.1,71.8,1000.7,2.5
6,02.06.25 - 18:00,14.1,85.3,1000.4,2.5
7,03.06.25 - 00:00,10.6,91.3,1002.0,2.1
8,03.06.25 - 06:00,10.7,90.3,1003.0,2.3
9,03.06.25 - 12:00,14.0,78.5,1004.2,2.7


SQL-spørringen over henter de første 10 radene der temperaturen er over 10. Den viser tidspunktet, temperaturen, fuktigheten, trykket og vindhastigheten.
- Dette hjelper med å se hvordan ulike varaibler oppfører seg sammen under forhold over 10 grader celcius.